In [43]:
from langchain_qdrant import Qdrant
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader

In [50]:
splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=100)

In [51]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

# List of files to load
pdf_files = [
    r"c:\Users\kingc\EY_final\Internal_DB\Company Docs.pdf",
    r"c:\Users\kingc\EY_final\Internal_DB\Minutes of Meeting Q3.pdf",
    r"c:\Users\kingc\EY_final\Internal_DB\Product Description Minocycline.pdf",
    r"c:\Users\kingc\EY_final\Internal_DB\Product Description Telmisartan.pdf"
]

docs = []
for file_path in pdf_files:
    # Check if file exists to avoid errors
    if os.path.exists(file_path):
        loader = PyMuPDFLoader(file_path=file_path)
        docs.extend(loader.load_and_split(text_splitter=splitter))
    else:
        print(f"Warning: File not found at {file_path}")

print(f"Successfully loaded {len(docs)} document pages.")

Successfully loaded 20 document pages.


In [62]:
embeddings=GoogleGenerativeAIEmbeddings(model='gemini-embedding-001',api_key=os.getenv("API_1"))

In [63]:
vectorstore = PineconeVectorStore.from_documents(index_name='boundless-alder', embedding=embeddings,documents=docs)

In [59]:
llm=init_chat_model(model='moonshotai/kimi-k2-instruct-0905',model_provider='groq',temperature=0.3)

In [60]:
result=embeddings.embed_query('hello world')

In [64]:
vectorstore=PineconeVectorStore(index_name='boundless-alder', embedding=embeddings)

In [65]:
vectorstore.similarity_search('What is cash cow of company?')

[Document(id='709904c8-587e-4186-8d60-a942418b8217', metadata={'author': '', 'creationDate': '', 'creationdate': '', 'creator': '', 'file_path': 'c:\\Users\\kingc\\EY_final\\Internal_DB\\Product Description Minocycline.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': '', 'moddate': '', 'page': 0.0, 'producer': 'Skia/PDF m144 Google Docs Renderer', 'source': 'c:\\Users\\kingc\\EY_final\\Internal_DB\\Product Description Minocycline.pdf', 'subject': '', 'title': 'Product Description Minocycline', 'total_pages': 2.0, 'trapped': ''}, page_content='oral acne antibiotics is shrinking. \n●\u200b Margin: High (due to fully depreciated manufacturing assets), but volume is the \nconstraint.'),
 Document(id='3997f1ad-7d54-4089-8d6c-b272ef518bb4', metadata={'author': '', 'creationDate': '', 'creationdate': '', 'creator': '', 'file_path': 'c:\\Users\\kingc\\EY_final\\Internal_DB\\Product Description Telmisartan.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': '', 'moddate': '', 'page': 0.0

In [66]:
from langchain.agents import create_agent


In [67]:
llm=init_chat_model(model='moonshotai/kimi-k2-instruct-0905',model_provider='groq',temperature=0.3)

In [68]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vectorstore.similarity_search(query, k=4)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
  """You are a senior pharmaceutical research analyst. Your task is to conduct deep and thorough web research on drug markets, competitor landscapes, and broader scientific context. Provide detailed, well-sourced answers strictly adhering to the user's requirements.  
"""
)
agent = create_agent(llm, tools, system_prompt=prompt)

: 

In [ ]:
agent.invoke({"messages": [{"role": "user", "content": "What are alternate uses of metformin"}]})